In [26]:
import os 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from glob import glob 
import json 

exp_list = [
# '/Volume/MoLeFlow/logs/baseline_v1.1_embed_dim_fix',
# '/Volume/MoLeFlow/logs/baseline_v1.1_embed_fix_jacobian_fix_no_ln_after_task0',
# '/Volume/MoLeFlow/logs/baseline_v1.2_embed_fix_jacobian_fix',
# '/Volume/MoLeFlow/logs/baseline_v1.3_embed_fix_jacobian_fix_soft_ln_init_scale_0.01',
# '/Volume/MoLeFlow/logs/baseline_v1.4_embed_fix_jacobian_fix_soft_ln_init_scale_0.01_lambda_logdet_1e-5',
# '/Volume/MoLeFlow/logs/baseline_v1.5_embed_fix_jacobian_fix_soft_ln_init_scale_0.01_lambda_logdet_1e-5_spatial_context_kernel_3',
# '/Volume/MoLeFlow/logs/baseline_v1.6_embed_fix_jacobian_fix_soft_ln_init_scale_0.01_scale_context_k3_a0.1',
# '/Volume/MoLeFlow/logs/baseline_v1.7_scale_context_k3_a0.1',
# '/Volume/MoLeFlow/logs/baseline_v1.8_patch_wise_context_gate_k3_hidden64',
'/Volume/MoLeFlow/logs/moleflow_run_tests',
'/Volume/MoLeFlow/logs/Version3-DIA_only',
'/Volume/MoLeFlow/logs/Version3-OGP_only',
'/Volume/MoLeFlow/logs/Version3-DIA_and_OGP',
'/Volume/MoLeFlow/logs/Version3-All',
'/Volume/MoLeFlow/logs/Version3-WhiteningAdapter_only',
'/Volume/MoLeFlow/logs/Version3-MSContext_only',
'/Volume/MoLeFlow/logs/Version3-WhiteningAdapter_DIA',
'/Volume/MoLeFlow/logs/Version3-WhiteningAdapter_OGP',
'/Volume/MoLeFlow/logs/Version3-WhiteningAdapter_DIA_OGP',
'/Volume/MoLeFlow/logs/Version3-MSContext_DIA',
'/Volume/MoLeFlow/logs/Version3-MSContext_OGP',
'/Volume/MoLeFlow/logs/Version3-MSContext_DIA_OGP'

]


df = pd.DataFrame()
for exp_path in exp_list:
    for log_path in glob(os.path.join(exp_path, 'final_results.csv')):
        temp_df = pd.read_csv(log_path)
        temp_df['exp_name'] = os.path.basename(exp_path)
        df = pd.concat([df, temp_df])





In [27]:
# The error is due to non-unique index, so we must reset the index for Styler to work.
df[df['Class Name']!=''].reset_index(drop=True).style.set_properties(
    subset=['exp_name'], **{'max-width': '600px', 'white-space': 'pre-wrap'}
)

,Task ID,Class Name,Routing Acc (%),Image AUC,Pixel AUC,Image AP,Pixel AP,exp_name
0,0,leather,100.000000,0.9990,0.9736,0.9996,0.1512,moleflow_run_tests
1,1,grid,100.000000,0.7293,0.8874,0.8696,0.1213,moleflow_run_tests
2,2,transistor,100.000000,0.7221,0.8888,0.6508,0.3579,moleflow_run_tests
3,Mean,Overall,100.000000,0.8168*,0.9166*,0.8400*,0.2101*,moleflow_run_tests
4,0,leather,100.000000,1.0000,0.9769,1.0000,0.1969,Version3-DIA_only
5,1,grid,100.000000,0.7652,0.8997,0.9052,0.1536,Version3-DIA_only
6,2,transistor,100.000000,0.7000,0.9065,0.6442,0.3866,Version3-DIA_only
7,Mean,Overall,100.000000,0.8217*,0.9277*,0.8498*,0.2457*,Version3-DIA_only
8,0,leather,100.000000,0.9993,0.9632,0.9998,0.1187,Version3-OGP_only
9,1,grid,100.000000,0.7277,0.8830,0.8583,0.1122,Version3-OGP_only


In [28]:
# The error is due to non-unique index, so we must reset the index for Styler to work.
df[df['Task ID']=='Mean'].reset_index(drop=True).style.set_properties(
    subset=['exp_name'], **{'max-width': '600px', 'white-space': 'pre-wrap'}
)

,Task ID,Class Name,Routing Acc (%),Image AUC,Pixel AUC,Image AP,Pixel AP,exp_name
0,Mean,Overall,100.000000,0.8168*,0.9166*,0.8400*,0.2101*,moleflow_run_tests
1,Mean,Overall,100.000000,0.8217*,0.9277*,0.8498*,0.2457*,Version3-DIA_only
2,Mean,Overall,100.000000,0.8180*,0.9161*,0.8417*,0.2105*,Version3-OGP_only
3,Mean,Overall,100.000000,0.8226*,0.9231*,0.8580*,0.2368*,Version3-DIA_and_OGP
4,Mean,Overall,100.000000,0.4471*,0.5527*,0.6540*,0.0898*,Version3-All
5,Mean,Overall,100.000000,0.7302*,0.8747*,0.8023*,0.1859*,Version3-WhiteningAdapter_only
6,Mean,Overall,100.000000,0.4323*,0.4066*,0.6353*,0.0784*,Version3-MSContext_only
7,Mean,Overall,100.000000,0.8614*,0.9351*,0.8829*,0.2756*,Version3-WhiteningAdapter_DIA
8,Mean,Overall,100.000000,0.4661*,0.4602*,0.6627*,0.0789*,Version3-MSContext_DIA
